In [30]:
pip install shutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for shutils: filename=shutils-0.1.0-py3-none-any.whl size=3283 sha256=216a26dc0a4e35b89745510cce814d7872de4f72a05cb1e123a4f604383e7b3a
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\42\9c\d4\ab0453013b2a9132735074a219123db607250a21d35ee3f616
Successfully built shutils
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install onnx

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
    --------------------------------------- 0.3/14.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.5 MB 1.9 MB/s eta 0:00:08
   -- ------------------------------------- 1.0/14.5 MB 2.0 MB/s eta 0:00:07
   --- ------------------------------------ 1.3/14.5 MB 2.0 MB/s eta 0:00:07
   ----- ---------------------------------- 2.1/14.5 MB 2.3 MB/s eta 0:00:06
   ------- -------------------------------- 2.6/14.5 MB 2.4 MB/s eta 0:00:05
   --------- ------------------------------ 3.4/14.5 MB 2.6 MB/s eta 0:00:05
   ---------- ----------------------------- 3.9/14.5 MB 2.7 MB/s eta 0:00:04
   ------------ --------------------------- 4.7/14.5 MB 2.9 MB/s eta 0:00:04
   --------------- ------------------------ 5.8/14.5 MB 3.0 MB/s eta 0:00:03
   ------------------ --------------------- 6.6/14.5 MB 3.1 MB/s eta 0:00:03
   -------------------- ------------------- 7.3/14.5 MB 3.2 MB/s eta 0:00:03
   ----------

In [3]:
# import library

import torch
import torch.onnx
import torch.nn as nn
import torch.optim as optim
import onnx
import torch.nn.functional as F
import tensorflow as tf
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
# The path to the main directory 
main_dir = "D:\\kaggle\\working\\new_directory" 

# List all classes in the main directory
classes = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]

# Print the names of the classes
print("Classes:")
for class_name in classes:
    print(class_name)

Classes:
Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___healthy
Cherry_(including_sour)___Powdery_mildew
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___healthy
Corn_(maize)___Northern_Leaf_Blight
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___healthy
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___healthy
Strawberry___Leaf_scorch
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___healthy
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_mosaic_virus
Tomato___Tomato_Yellow_Leaf_Curl_Virus


# Data Processing

In [5]:
dataset_path = 'D:\\kaggle\\working\\new_directory'

In [6]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),              # resize all images to 150x150 pixels
    transforms.ToTensor(),                      # convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [8]:
dataset.classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

# Train-Test split

In [10]:
train_idx,test_idx = train_test_split(list(range(len(dataset))),test_size=0.3,random_state=42)

train_dataset = Subset(dataset,train_idx)
test_dataset = Subset(dataset,test_idx)

In [11]:
trainloader = DataLoader(train_dataset,batch_size=15,shuffle=True)
testloader = DataLoader(test_dataset,batch_size=15,shuffle=True)

# Define CNN Model

In [52]:
class CNN_Classification(nn.Module):
    def __init__(self):
        super(CNN_Classification,self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(3,32,kernel_size=3,stride=1,padding=1)  # first Convolutional layer
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)     # first pooling layer
        self.conv2 = nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1) # second Convolution layer
        self.fc1 = nn.Linear(64*37*37,128)                             # 37 x 37 is the size after pooling
        self.fc2 = nn.Linear(128, len(dataset.classes))

    def forward(self,x):
         x = self.pool(F.relu(self.conv1(x)))
         x = self.pool(F.relu(self.conv2(x)))
         x = torch.flatten(x,1)                        # flatten all dimensions
         x = torch.relu(self.fc1(x))
         x = self.fc2(x)
         return x
         

In [53]:
CNN_Model  = CNN_Classification()

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_Model.parameters(),lr=0.001)         #Defines the optimizer, Adam with a learning rate of 0.001

# Train the Model

In [55]:
num_epochs = 10
for epoch in range(num_epochs):

    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = CNN_Model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training')

Epoch 1/10, Loss: 0.8594730496406555
Epoch 2/10, Loss: 0.368681401014328
Epoch 3/10, Loss: 0.07125569880008698
Epoch 4/10, Loss: 0.501944899559021
Epoch 5/10, Loss: 0.0022129875142127275
Epoch 6/10, Loss: 0.5597519874572754
Epoch 7/10, Loss: 0.0037620356306433678
Epoch 8/10, Loss: 0.0032458750065416098
Epoch 9/10, Loss: 0.6228703260421753
Epoch 10/10, Loss: 0.22277356684207916
Finished Training


# Evaluation

In [56]:
import torch
from sklearn.metrics import precision_score,recall_score,f1_score

# Switch to evaluation mode
CNN_Model.eval()

#Initialize variable for storing predictions and true labels
all_preds = []
all_labels = []

#correct and total for accuracy
correct=0
total=0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images,labels = data

        # Forward pass to get predictions
        outputs = CNN_Model(images)
        _,predicted = torch.max(outputs.data,1)

        # update accuracy calculations
        total +=labels.size(0)
        correct +=(predicted == labels).sum().item()

        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct/total
print(f'Accuracy: {accuracy:.2f} %')

#Calculate precision,recall,and F1_score using sklearn
precision = precision_score(all_labels,all_preds,average = 'weighted')
recall = recall_score(all_labels,all_preds,average = 'weighted')
f1 = f1_score(all_labels,all_preds,average = 'weighted')

print(f'Precision: {precision * 100:.4f}%')
print(f'Recall: {recall * 100:.4f}%')
print(f'F1-score: {f1 * 100:.4f}%')


Accuracy: 87.19 %
Precision: 87.7547%
Recall: 87.1876%
F1-score: 87.2259%


# Prediction

In [60]:
# Load a test image and preprocess it
img = Image.open("D:\\GUVI AI & ML\\Capstone_Final_Project2_Plant Disease Detection from Images\\dataset\\New Plant Diseases Dataset(Augmented)\\test\\test\\TomatoHealthy3.JPG")
img = transform(img).unsqueeze(0)  # add batch dimension

# pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)

_,predicted = torch.max(output,1)
print(f'Predicted Class: {dataset.classes[predicted.item()]}')


Predicted Class: Tomato___Target_Spot


In [61]:
# Load a test image and preprocess it
img = Image.open("D:\\GUVI AI & ML\\Capstone_Final_Project2_Plant Disease Detection from Images\\dataset\\New Plant Diseases Dataset(Augmented)\\test\\test\\PotatoEarlyBlight2.JPG")
img = transform(img).unsqueeze(0)  # add batch dimension

# pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)

_,predicted = torch.max(output,1)
print(f'Predicted Class: {dataset.classes[predicted.item()]}')

Predicted Class: Potato___Early_blight


In [62]:
# Load a test image and preprocess it
img = Image.open("D:\\GUVI AI & ML\\Capstone_Final_Project2_Plant Disease Detection from Images\\dataset\\New Plant Diseases Dataset(Augmented)\\test\\test\\AppleCedarRust3.JPG")
img = transform(img).unsqueeze(0)  # add batch dimension

# pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)

_,predicted = torch.max(output,1)
print(f'Predicted Class: {dataset.classes[predicted.item()]}')

Predicted Class: Apple___Cedar_apple_rust


In [63]:
torch.save(CNN_Model.state_dict(), 'cnn_model.pth')

In [65]:
CNN_Model.load_state_dict(torch.load("D:\\GUVI AI & ML\\Capstone_Final_Project2_Plant Disease Detection from Images\\cnn_model.pth"))

<All keys matched successfully>

# Pretrained Models

# VGG16

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained VGG16 Models
vgg16 = models.vgg16(pretrained=True)

# transfer model to the GPU
vgg16 = vgg16.to(device)

# Freeze all the layers
for param in vgg16.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
vgg16.classifier[6] = nn.Linear(4096, len(dataset.classes)) #4096 is oinput to the last layer

# Transfer classifier changes to the GPU
vgg16.classifier = vgg16.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device) # move criterion to GPU
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

# Train the model

num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = vgg16(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training VGG16')

Using device: cpu


C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5, Loss: 1.0485159158706665
Epoch 2/5, Loss: 0.09223437309265137
Epoch 3/5, Loss: 0.7216312885284424
Epoch 4/5, Loss: 0.020727457478642464
Epoch 5/5, Loss: 0.9807632565498352
Finished Training VGG16


In [ ]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation mode
vgg16.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = vgg16(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.4f}%')
print(f'Recall: {recall * 100:.4f}%')
print(f'F1-score: {f1 * 100:.4f}%')


# AlexNet

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Transfer model to the GPU
alexnet = alexnet.to(device)

# Freeze layers if desired
for param in alexnet.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
alexnet.classifier[6] = nn.Linear(4096, len(dataset.classes))

# Transfer classifier changes to the GPU
alexnet.classifier = alexnet.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move loss function to GPU
optimizer = optim.Adam(alexnet.parameters(), lr=0.02)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training AlexNet')

Using device: cpu
Epoch 1/5, Loss: 19.385351181030273
Epoch 2/5, Loss: 0.0
Epoch 3/5, Loss: 0.0
Epoch 4/5, Loss: 0.0
Epoch 5/5, Loss: 77.12576293945312
Finished Training AlexNet


In [13]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation model
alexnet.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = alexnet(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')

Accuracy: 87.63 %
Precision: 88.82%
Recall: 87.63%
F1-score: 87.72%


# DenseNet

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained DenseNet121 model
densenet = models.densenet121(pretrained=True)

# Transfer model to the GPU
densenet = densenet.to(device)

# Freeze all the layers if you don't want to train the convolutional layers (optional)
for param in densenet.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
num_ftrs = densenet.classifier.in_features
densenet.classifier = nn.Linear(num_ftrs, len(dataset.classes))

# Transfer classifier changes to the GPU
densenet.classifier = densenet.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move loss function to GPU
optimizer = optim.Adam(densenet.parameters(), lr=0.02)

# Train the model
num_epochs = 4
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = densenet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training DenseNet')

Using device: cpu


C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\Lenovo/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|█████████████████████████████████████████████████████████████████████████████| 30.8M/30.8M [00:18<00:00, 1.74MB/s]


Epoch 1/4, Loss: 3.9687535762786865
Epoch 2/4, Loss: 3.110856056213379
Epoch 3/4, Loss: 1.5293303728103638
Epoch 4/4, Loss: 0.0
Finished Training DenseNet


In [15]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation model
# densenet.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = densenet(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')

Accuracy: 88.23 %
Precision: 90.14%
Recall: 88.23%
F1-score: 88.34%


In [19]:
torch.save(vgg16.state_dict(), 'vgg16.pth')  
torch.save(alexnet.state_dict(), 'alexnet_model.pth') 
torch.save(densenet.state_dict(), 'densenet.pth')